In [67]:
import numpy as np
from dotenv import load_dotenv
from dwave.system import DWaveSampler, EmbeddingComposite
from numpy import hamming
from openjij import SQASampler
from pyqubo import Array, Constraint, LogEncInteger, solve_qubo
import math
import os
import dimod

In [68]:
load_dotenv()

TOKEN = os.getenv("TOKEN")
endpoint = "https://cloud.dwavesys.com/sapi/"

In [69]:
W = 750
cost = [135,
        139,
        149,
        150,
        156,
        163,
        173,
        184,
        192,
        201,
        210,
        214,
        221,
        229,
        240, ]
weight = [70,
          73,
          77,
          80,
          82,
          87,
          90,
          94,
          98,
          106,
          110,
          113,
          115,
          118,
          120, ]
opt = [0,
1,
0,
1,
0,
0,
1]
c = {}
w = {}
N = len(cost)
for i in range(N):
    c[i] = cost[i]
    w[i] = weight[i]

In [70]:
x = Array.create('x', shape=(N), vartype='BINARY')
y = LogEncInteger("y", (0, W))

In [71]:
key1 = max(c, key=lambda k: c[k])
B = 1
A = B * c[key1] + 1

In [72]:
HA = Constraint(
    A * (W - sum(w[a] * x[a] for a in range(N)) - y)**2, label='HA'
)

HB = - B * sum(c[a] * x[a] for a in range(N))

In [73]:
H = HA + HB
model = H.compile()
Q, offset = model.to_qubo()

In [74]:
dw_sampler = DWaveSampler(solver='DW_2000Q_6',
                          token=TOKEN, endpoint=endpoint)
sampler = EmbeddingComposite(dw_sampler)
sampleset = sampler.sample_qubo(Q, num_reads=30)

In [75]:
sampleset.record

rec.array([([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0], -1.37239159e+08, 1, 0.16),
           ([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0], -1.37250871e+08, 1, 0.16),
           ([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0], -1.20245291e+08, 1, 0.16),
           ([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0], -1.37250871e+08, 1, 0.16),
           ([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0], -1.21385015e+08, 1, 0.16),
           ([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0], -1.37250139e+08, 1, 0.16),
           ([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0], -1.37098615e+08, 1, 0.16),
           ([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0], -1.37180599e+08, 2, 0.16),
           ([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,

In [76]:
decoded_sample = model.decode_sample(sampleset.first.sample, vartype="BINARY")

In [77]:
cost = 0
weight = 0
for k in range(N):
    if decoded_sample.array('x', k) != 0:
        cost += c[k]
        weight += w[k]

In [78]:
sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem))
            for elem in range(math.ceil(math.log2(W)))])

In [79]:
print("スラック変数Y = {}".format(sol_y))
print()
print("broken")
print(decoded_sample.constraints(only_broken=True))
print("cost : "+str(cost))
print("weight : "+str(weight))

スラック変数Y = 165

broken
{}
cost : 1115
weight : 585


In [80]:
init = sampleset.samples()[-1]

最悪値

In [81]:
sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem))
            for elem in range(math.ceil(math.log2(W)))])

In [82]:
t = (0, 5, 20)
s = (1.0, 0.4, 1.0)

schedule = list(zip(t, s))

In [83]:
dw_sampler = DWaveSampler(solver='DW_2000Q_6',
                          token=TOKEN, endpoint=endpoint)
sampler = EmbeddingComposite(dw_sampler)
sampleset = sampler.sample_qubo(
    Q, num_reads=30, anneal_schedule=schedule, initial_state=init)

In [84]:
sampleset.record

rec.array([([1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1], -1.27096935e+08, 1, 0.2 ),
           ([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1], -1.31760899e+08, 1, 0.12),
           ([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1], -1.15872595e+08, 1, 0.12),
           ([1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1], -1.23661231e+08, 1, 0.12),
           ([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1], -1.17292675e+08, 1, 0.08),
           ([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1], -1.37031310e+08, 1, 0.16),
           ([0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1], -1.34298502e+08, 1, 0.08),
           ([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1], -1.30688275e+08, 1, 0.12),
           ([1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,

In [85]:
decoded_sample = model.decode_sample(sampleset.first.sample, vartype="BINARY")

In [86]:
cost = 0
weight = 0
for k in range(N):
    if decoded_sample.array('x', k) != 0:
        cost += c[k]
        weight += w[k]

In [87]:
sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem))
            for elem in range(math.ceil(math.log2(W)))])

In [88]:
print("スラック変数Y = {}".format(sol_y))
print()
print("broken")
print(decoded_sample.constraints(only_broken=True))
print("cost : "+str(cost))
print("weight : "+str(weight))

スラック変数Y = 675

broken
{'HA': (False, 35136.0)}
cost : 697
weight : 360
